In [1]:
import sys, os
sys.path.append(os.path.abspath("../src"))

import importlib
import simulator, entries, exits, sizers
importlib.reload(simulator); importlib.reload(entries); importlib.reload(exits); importlib.reload(sizers)

from simulator import Simulator, FillModel
from entries import random_entry
from exits import time_stop_exit
from sizers import flat_sizer


ImportError: cannot import name 'random_entry' from 'entries' (C:\exits-lab\src\entries.py)

In [ ]:
# Make src/ importable
import sys, os
sys.path.append(os.path.abspath("../src"))

import numpy as np
import pandas as pd

# Load the feature parquet you just saved
bars = pd.read_parquet("../data/eurusd_m1_features.parquet")

# Keep only what's needed for the sim loop for now
bars = bars[['open','high','low','close']].copy()
bars = bars.sort_index()
print(bars.shape, bars.index.min(), "→", bars.index.max())
display(bars.head())


In [ ]:
from simulator import Simulator, FillModel
from entries import random_entry
from exits import time_stop_exit
from sizers import flat_sizer

# small wrapper so we can push time_stop_bars into the position meta
def flat_sizer_with_time_stop(equity, series_state, signal, cfg):
    size, meta = flat_sizer(equity, series_state, signal, cfg)
    # put the time stop into meta so exits.time_stop_exit can read it
    meta = dict(meta)
    meta['time_stop_bars'] = cfg.get('time_stop_bars', 30)  # 30 bars ~ 30 minutes on M1
    return size, meta

# config for the run
cfg = {
    'start_equity': 10_000.0,
    'latency_bars': 1,        # act on next bar open (no look-ahead)
    'risk_pct': 0.004,        # used by flat_sizer
    'time_stop_bars': 30,     # <-- try 15/30/60 later
}

# crude fill model (we’ll refine later; spread/slippage handled via exits later if needed)
fill = FillModel(spread_pips=0.8, slippage_pips=0.2)

# build simulator
sim = Simulator(
    bars=bars,
    cfg=cfg,
    entry_fn=random_entry,             # random entries
    exit_fn=time_stop_exit,            # time-based exit only for this first smoke test
    sizer_fn=flat_sizer_with_time_stop,
    fill=fill
)

trades = sim.run(seed=42)
print(trades.head())
print("Num trades:", len(trades))


In [ ]:
from evaluation import expectancy, win_rate

if len(trades):
    print("Expectancy (avg pnl/trade):", round(expectancy(trades), 6))
    print("Win rate:", round(win_rate(trades)*100, 2), "%")
    print(trades['reason'].value_counts())
else:
    print("No trades produced — increase random entry frequency or run on a longer date range.")
